In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,431 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRele

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-10-09 04:42:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-10-09 04:42:14 (6.60 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Vine_Analysis").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# Store environmental variable
from getpass import getpass
password = getpass('Enter database password')

# Configure settings for RDS
jdbc_url="jdbc:postgresql://amazon-vine.cgecvncx2stq.us-east-2.rds.amazonaws.com:5432/vine_analysis"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [5]:
from pyspark.sql import DataFrameReader
# Read postgres table into dataframe
vine_df = spark.read.jdbc(url=jdbc_url, table='vine_table', properties=config)

In [6]:
# Filter only reviews with more than 20 votes
high_vote_vine_df = vine_df.filter('total_votes>=20')

In [7]:
# Filter only reviews with high ratio of helpful votes to total votes
helpful_vote_vine_df = high_vote_vine_df.withColumn('percentage', high_vote_vine_df['helpful_votes'] / high_vote_vine_df['total_votes'] ).filter('percentage>.5').drop('percentage')
helpful_vote_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1EADU8IU77GOT|          2|           18|         25|   N|                Y|
|R2UZIW1QI38IUH|          3|           20|         27|   N|                N|
|R15X51D8MMOYUU|          5|           42|         46|   N|                Y|
|R2NJ7SNM4X63TK|          5|           57|         63|   N|                Y|
|R1IXFPCQG71W0Q|          3|           18|         20|   N|                Y|
|R22E9TZRA29WZE|          5|           24|         26|   N|                N|
|R3LX8T8FW3GFJJ|          5|           17|         24|   N|                Y|
| R9REJKVBANRN6|          2|           14|         20|   N|                N|
|R12T2CW1VAKJLP|          1|           16|         23|   N|                Y|
| RIUKST053BR2K|          5|           25|         26|   Y|     

In [8]:
# Filter for paid vine program reviews
paid_vine_df = helpful_vote_vine_df.filter('vine=="Y"')
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RIUKST053BR2K|          5|           25|         26|   Y|                N|
|R1IN4WPVLZB86G|          5|           34|         37|   Y|                N|
|R1W1YY3QZDVKK3|          2|           26|         36|   Y|                N|
|  RV8FVXVWTEAX|          4|           97|        107|   Y|                N|
|R3CMWDEYCJM2O9|          1|           24|         31|   Y|                N|
|R2INH817LTJFGI|          5|           18|         20|   Y|                N|
|R10P2XVXA9DYTO|          5|           21|         25|   Y|                N|
| RW25NTS59UG55|          4|           39|         43|   Y|                N|
| RKKMH184GEHKD|          5|           19|         23|   Y|                N|
| RNRUIQVK27RDZ|          3|           17|         25|   Y|     

In [9]:
# Filter for unpaid vine program reviews
unpaid_vine_df = helpful_vote_vine_df.filter('vine=="N"')
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1EADU8IU77GOT|          2|           18|         25|   N|                Y|
|R2UZIW1QI38IUH|          3|           20|         27|   N|                N|
|R15X51D8MMOYUU|          5|           42|         46|   N|                Y|
|R2NJ7SNM4X63TK|          5|           57|         63|   N|                Y|
|R1IXFPCQG71W0Q|          3|           18|         20|   N|                Y|
|R22E9TZRA29WZE|          5|           24|         26|   N|                N|
|R3LX8T8FW3GFJJ|          5|           17|         24|   N|                Y|
| R9REJKVBANRN6|          2|           14|         20|   N|                N|
|R12T2CW1VAKJLP|          1|           16|         23|   N|                Y|
|R3416Q1IJEW3DZ|          2|           15|         22|   N|     

In [11]:
# Calculate Total Reviews
total_paid_reviews = paid_vine_df.count()
total_unpaid_reviews = unpaid_vine_df.count()

In [14]:
# Calculate total 5 star reviews
total_paid_5_star = paid_vine_df.filter('star_rating == 5').count()
total_unpaid_5_star = unpaid_vine_df.filter('star_rating == 5').count()

In [21]:
# Calculate Percentage of 5 star Reviews to Total
total_5_star_percent_paid = (total_paid_5_star / total_paid_reviews * 100)
total_5_star_percent_unpaid = (total_unpaid_5_star / total_unpaid_reviews * 100)

In [27]:
results_df = spark.createDataFrame([
                                    ('Vine', total_paid_reviews, total_paid_5_star, f'{round(total_5_star_percent_paid, 2)}%') ,
                                    ('Non-Vine', total_unpaid_reviews, total_unpaid_5_star, f'{round(total_5_star_percent_unpaid, 2)}%')
], ['Reviewer Type', 'Total Reviews', 'Total 5 Star Reviews', 'Total Percentage 5 Star vs Total'])
results_df.show()

+-------------+-------------+--------------------+--------------------------------+
|Reviewer Type|Total Reviews|Total 5 Star Reviews|Total Percentage 5 Star vs Total|
+-------------+-------------+--------------------+--------------------------------+
|         Vine|           94|                  48|                          51.06%|
|     Non-Vine|        39915|               15556|                          38.97%|
+-------------+-------------+--------------------+--------------------------------+

